# Les vues matérialisées

Cette note décrit comment ajouter une nouvelle vue matérialiée ou mettre à jour une vue existante.

Une vue matérailisée n'est vavlable que pour un site.
Pour créer une nouvelle vue, vous pouvez utiliser soit une requête SQL brute, soit un Queryset Django.

La marche à suivre est la suivante:
- ajouter un fichier python ou SQL dans `recoco/apps/metrics/sql_queries`, qui contient la requête SQL permettant de créer la vue,
    (NB: la requête doit prendre en paramètre le `site_id`)
- éditer les specs de la vue dans les settings `MATERIALIZED_VIEWS_SPEC`,
- recréer les vues pour tous les sites avec la commande Django `update_materialized_views`.

Le nom de la vue en base suivra le pattern suivant: `{settings.MATERIALIZED_VIEWS_PREFIX}_{self.site_id}_{self.name}`.
Exemple pour une vue nommée "ma_vue" sur le site numero 9: `mv_9_ma_vue`.

In [ ]:

-- Exemple de requête SQL brute, avec le site_id en paramètre
-- Créer un fichier contenant la requête SQL dans `recoco/apps/metrics/sql_queries/ma_vue_rawsql.sql`

SELECT "projects_project"."id",
    COUNT("tasks_task"."id") AS "task_count"
FROM "projects_project"
    INNER JOIN "projects_project_sites" ON (
        "projects_project"."id" = "projects_project_sites"."project_id"
    )
    LEFT OUTER JOIN "tasks_task" ON (
        "projects_project"."id" = "tasks_task"."project_id"
    )
WHERE (
        "projects_project"."deleted" IS NULL
        AND "projects_project_sites"."site_id" = %s
    )
GROUP BY "projects_project"."id"

In [ ]:
# La même requête que précédemment, mais cette fois-ci en utilisant un Queryset Django
# Créer un fichier contenant la code Django retournant le Queryset dans `recoco/apps/metrics/sql_queries/ma_vue_queryset.py`

from recoco.apps.projects.models import Project
from django.db.models import Count, QuerySet


def get_queryset(site_id: int) -> QuerySet:
    return (
        Project.objects.filter(sites__id=site_id)
        .values("id")
        .annotate(task_count=Count("tasks"))
    )


In [ ]:
# Mettre à jour les settings, afin de référencer les vues nouvellement créées
# On spécifie égaleement ici les index qu'on souhaite créer sur ces vues

MATERIALIZED_VIEWS_SPEC = [
    {
        "name": "ma_vue_rawsql",
        "indexes": [],
        "unique_indexes": [],
    },
    {
        "name": "ma_vue_queryset",
        "indexes": [],
        "unique_indexes": [],
    },
]

In [ ]:
# Recréer les vues materialisées pour chaque site et les mettre à jour avec cette commande

python manage.py update_materialized_views